### **Data Transformation (or data augmentation) as per the AlexNet paper** ##

data augmentation (only image translation and horizontal reflection; read section 4.1 
on the paper)
- Random crop of 224x224 from the 256x256 image
- Random horizontal flipping

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import random

In [3]:
# Setting random seeds for reproducibility
np.random.seed(42)
random.seed(42)

In [4]:
import os

# Define dataset paths
dataset_root = 'alexnet_dataset'
train_dir = os.path.join(dataset_root, 'train')
val_dir = os.path.join(dataset_root, 'val')
test_dir = os.path.join(dataset_root, 'test')

#### Why Class mapping? ####

In [5]:
# Load class mapping
class_to_idx = {}
with open(os.path.join(dataset_root, 'class_mapping.txt'), 'r') as f:
    for line in f:
        cls, idx = line.strip().split(',')
        class_to_idx[cls] = int(idx)

### Define Transformation for datasets ###

In [6]:
# Data transformation for training
train_transform = transforms.Compose([
    transforms.RandomCrop(224),  # Random crop to 224x224
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.ToTensor(),  # Convert to tensor (scales to [0, 1])
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet normalization

])

# Data transformation for validation/testing (no augmentation)
val_transform = transforms.Compose([
    transforms.CenterCrop(224),  # Center crop to 224x224
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])